In [0]:
from keras.models import model_from_yaml
from keras.utils.data_utils import get_file
from keras.optimizers import Adam
import numpy as np
import urllib.request
import json

Using TensorFlow backend.


In [0]:
with urllib.request.urlopen("https://people.ucsc.edu/~gcollelu/generator.h5") as url:
    generator_weights = url.read()
with urllib.request.urlopen("https://people.ucsc.edu/~gcollelu/generator.yaml") as url:
    generator_model = url.read()

In [0]:
with urllib.request.urlopen("https://people.ucsc.edu/~gcollelu/flat_steps.json") as url:
    flat_steps = json.loads(url.read().decode())
with urllib.request.urlopen("https://people.ucsc.edu/~gcollelu/flat_ingredients.json") as url:
    flat_ingredients = json.loads(url.read().decode())

## Decode The Generated Matrix

Once we have a matrix generated, we need to decode it.

In [0]:
def deprocess(x):
    x = (x / 2)
    x += x + 0.5
    x = x*8425231679608
    x = np.clip(x, 0, 8425231679608)
    x = np.uint64(x)
    x = x.reshape(15, 20)
    return x

def decode_triplet(encoded_triplet):
    encoded_triplet = int(encoded_triplet)
    position = encoded_triplet % 100
    encoded_triplet =  encoded_triplet // 100
    step = encoded_triplet % 1000000
    encoded_triplet = encoded_triplet // 1000000
    ingredient = str(encoded_triplet)
    return (ingredient, step, position)

def decode_recipe_matrix(matrix):
    rows = matrix.shape[0]
    cols = matrix.shape[1]
    recipe_encoded = {}
    
    for i in range(0, rows):
        for j in range(0, cols):
            if matrix[i][j] < 0.5 and matrix[i][j] > -0.5 :
                continue
            (ingredient, step, position) = decode_triplet(matrix[i][j])
            if not position == 0:  
                if ingredient not in recipe_encoded:
                    recipe_encoded[ingredient] = []
                recipe_encoded[ingredient].append([step, position])
    return recipe_encoded

def decode_recipe(encoded_recipe, flat_ingredients, flat_steps):
    decoded_recipe = {}
    decoded_recipe["ingredients"] = set([])
    decoded_recipe["steps"] = set([])
    for ingredient_id in encoded_recipe:
        ingredient_id = str(int(ingredient_id) % 84253)
        decoded_recipe["ingredients"].add(flat_ingredients[int(ingredient_id)])
        for (step_id, position) in encoded_recipe[ingredient_id]:
            step_id = step_id % 452362
            try:
                decoded_recipe["steps"].add((flat_steps[step_id], position))
            except IndexError:
                print(step_id)
    
    decoded_recipe["ingredients"] = list(decoded_recipe["ingredients"])
    decoded_recipe["steps"] = list(decoded_recipe["steps"])
    decoded_recipe['steps'] =  sorted(decoded_recipe['steps'], key=lambda tup: tup[1])
    decoded_recipe['steps'] = [ seq[0] for seq in decoded_recipe['steps'] ]

    return decoded_recipe

def full_decode_to_original(matrix_recipe, flat_ingredients, flat_steps):
    matrix_recipe = deprocess(matrix_recipe)
    decoded_matrix_json = decode_recipe_matrix(matrix_recipe)
    original_recipe = decode_recipe(decoded_matrix_json, flat_ingredients, flat_steps)
    return original_recipe

## Load Model & Weights

We load the previously trained model and weights

In [0]:
generator = model_from_yaml(generator_model)
weights_path = get_file('generator.h5','https://people.ucsc.edu/~gcollelu/generator.h5')
generator.load_weights(weights_path)

Instructions for updating:
Colocations handled automatically by placer.
221184/219680 [==============================] - 1s 3us/step


In [0]:
generator.compile(optimizer=Adam(lr=0.01), loss='binary_crossentropy')

In [0]:
def make_latent_samples(sample_size):
    #return np.random.uniform(-1, 1, size=(n_samples, sample_size))
    return np.random.normal(loc=0, scale=1, size=(1, sample_size))

## Generate New Recipe

Now, we generate a new recipe

In [0]:
latent_samples = make_latent_samples(100)
generated_recipe = generator.predict(latent_samples)
original_recipe = full_decode_to_original(generated_recipe, flat_ingredients, flat_steps)
print(json.dumps(original_recipe, indent=2))

{
  "ingredients": [
    "head green cabbage, chopped",
    "ruby port",
    "slices bacon, cooked crisp and crumbled, divided",
    "seedless orange, peeled, chopped",
    "blue cheese crumbles, divided"
  ],
  "steps": [
    "Slice chicken breasts on the diagonal into 1/2 inch thick slices",
    "Divide dough into 4 equal portions",
    "Spoon onto the toasted baguette rounds",
    "Process until mixture is bright green and smooth",
    "Bake the kale for 10 minutes, watching carefully to prevent burning, until the kale is crisp and the cheese is browned"
  ]
}
